<img src="https://i.imgur.com/6U6q5jQ.png"/>

_____
<a id='home'></a>


<a target="_blank" href="https://colab.research.google.com/github/SocialAnalytics-StrategicIntelligence/codes/blob/main/Intro_To_Optimization.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Introduction to Optimization


In [ ]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vQHq0p2eTmxRWJjDmo1mUmdarYgIrEew4ieiVbIGQy-D_CyBw5rbbRUlRxwLKKaVQpRV9Hs8MGnz0X2/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>


# Part 1: Solving the Problem

Please, go to your _environment_ in Anacoda Navigator to install **glpk** and **pulp**  before runing the codes below.
Then, call the library:

In [ ]:
import pulp as pp

1. **Initialize the MODEL**: just write the name and declare if it is maximization or minimization problem type.

In [ ]:
model = pp.LpProblem(name='refinery-problem', # just the name
                     sense=pp.LpMaximize) # type of problem

2. **Declare the VARIABLES**: The refinery model consists of these _variables_:

In [ ]:
# how much gas?
Gas = pp.LpVariable(name="Gas",  # just the name
                    lowBound=0,  # ensure non-negativity
                    cat='Continuous') # here: you accept decimal values

# how much oil?
Oil = pp.LpVariable(name="Oil",
                 lowBound=0,
                 cat='Continuous')

3. **Create function to OPTIMIZE**: The function is just the linear combination of the variables and their _given coefficients__: 

In [ ]:
GasCoeff=1.9
OilCoeff=1.5
obj_func = GasCoeff*Gas + OilCoeff*Oil

4. **Represent the constraints**: These are the rules the model (set of variables) must obey:

In [ ]:
# SUBJECT TO:
C1= pp.LpConstraint(name='Gas Constraint',   # just the name
                    e= 1*gas - 2*oil, rhs=0, # linear combination of constraint and rhs 
                    sense=pp.LpConstraintGE) # 'rule' >= 0 (LpConstraintGE)
C2= pp.LpConstraint(name='Oil Constraint',
                    e= 1*oil, rhs=3000000,
                    sense=pp.LpConstraintGE) # 'rule' >= 3000000 (LpConstraintGE)
C3= pp.LpConstraint(name='Demand Constraint',
                    e= 1*gas, rhs=6400000,
                    sense=pp.LpConstraintLE, )# 'rule' <= 6400000 (LpConstraintLE)

5. **Build MODEL**: Here you add (i) the objective function, and (ii) all the constraints:

In [ ]:
model += obj_func
model += C1
model += C2
model += C3


6. **Solve the MODEL**: Notice we are not using the _default solver_, we are explicitly usig **COIN_CMD**:

In [ ]:
solver = pp.COIN_CMD(msg=False)
result=model.solve(solver)

# Part 2: Seeing optimal solution

You can create a summary like this:

In [ ]:
Results={"Model Status":pp.LpStatus[model.status]}
Results.update({"Optimal Solution":pp.value(model.objective)})
Results.update({v.name: v.varValue for v in model.variables()})
pd.DataFrame.from_dict(Results,orient='index').T.set_index('Model Status').style.format('{:,}')

# Part 3: Sensibility of the Solution

## Computing Shadow and Slack

The solution obtained comes with two extra pieces of information:

* The **slack**: The amount you can modify in the RHS, so that the current Optimal Result does not change. 
* The **shadow price**: The change in the Optimal result, if you change the RHS in one unit.

In [ ]:
import pandas as pd

pd.DataFrame([{'name':name,'slack': c.slack,'shadow price':c.pi} for name, c in model.constraints.items()])

## Using Slack

That is, if currently the RHS of 'oil' constraint is 3,000,000, if we add its slack (200,000) or less.

In [ ]:
model = pp.LpProblem(name='refinery-problem',sense=pp.LpMaximize)

C1= pp.LpConstraint(name='Gas Constraint',e= 1*gas - 2*oil, rhs=0,sense=pp.LpConstraintGE) 
C2= pp.LpConstraint(name='Oil Constraint',e= 1*oil, rhs=3200000,sense=pp.LpConstraintGE) # use SLACK
C3= pp.LpConstraint(name='Demand Constraint',e= 1*gas, rhs=6400000,sense=pp.LpConstraintLE, )

model += obj_func
model += C1
model += C2
model += C3

solver = pp.COIN_CMD(msg=False)
result=model.solve(solver)

Results={"Model Status":pp.LpStatus[model.status]}
Results.update({"Optimal Solution":pp.value(model.objective)})
Results.update({v.name: v.varValue for v in model.variables()})
pd.DataFrame.from_dict(Results,orient='index').T.set_index('Model Status').style.format('{:,}')

## Using Shadow price

As the shadow price of _Demand_ is 2.65, this will happen to the optimal solution if we add __1__ to the RHS of _Demand_:

In [ ]:
model = pp.LpProblem(name='refinery-problem',sense=pp.LpMaximize)

C1= pp.LpConstraint(name='Gas Constraint',e= 1*gas - 2*oil, rhs=0,sense=pp.LpConstraintGE) 
C2= pp.LpConstraint(name='Oil Constraint',e= 1*oil, rhs=3000000,sense=pp.LpConstraintGE) 
C3= pp.LpConstraint(name='Demand Constraint',e= 1*gas, rhs=6400001,sense=pp.LpConstraintLE) # add 1 (use shadow price)

model += obj_func
model += C1
model += C2
model += C3

solver = pp.COIN_CMD(msg=False)
result=model.solve(solver)

Results={"Model Status":pp.LpStatus[model.status]}
Results.update({"Optimal Solution":pp.value(model.objective)})
Results.update({v.name: v.varValue for v in model.variables()})
pd.DataFrame.from_dict(Results,orient='index').T.set_index('Model Status').style.format('{:,}')

# More Examples

## The diet problem

In [ ]:
%%html

<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vTSq9X74urGAB_5n_MIJ9ZGIboKSvBdokVTBXVLh_qqZnmLRTJioOF431Rzys3Qi9UaFwWXjeq6Wmd5/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

In [ ]:
model = pp.LpProblem(name='diet-problem', sense=pp.LpMinimize)

V_V = pp.LpVariable(name="Vega Vita", lowBound=0,cat='Integer')
H_H = pp.LpVariable(name="Happy Health", lowBound=0,cat='Integer')

obj_func = 0.2*V_V + 0.3*H_H

C1=pp.LpConstraint(e=20*V_V + 30*H_H,sense=pp.LpConstraintGE,name='Vitamin C',rhs=60)
C2=pp.LpConstraint(e=500*V_V + 250*H_H,sense=pp.LpConstraintGE,name='Calcium',rhs=1000)
C3=pp.LpConstraint(e=9*V_V + 2*H_H,sense=pp.LpConstraintGE,name='Iron',rhs=18)
C4=pp.LpConstraint(e=2*V_V + 10*H_H,sense=pp.LpConstraintGE,name='Niacin',rhs=20)
C5=pp.LpConstraint(e=60*V_V + 90*H_H,sense=pp.LpConstraintGE,name='Magnesium',rhs=360)


model += obj_func
model += C1
model += C2
model += C3
model += C4
model += C5

solver = pp.COIN_CMD(msg=False)
result=model.solve(solver)

Results={"Model Status":pp.LpStatus[model.status]}
Results.update({"Optimal Solution":pp.value(model.objective)})
Results.update({v.name: v.varValue for v in model.variables()})
pd.DataFrame.from_dict(Results,orient='index').T.set_index('Model Status').style.format('{:,}')

In [ ]:
pd.DataFrame([{'name':name,'slack': c.slack,'shadow price':c.pi} for name, c in model.constraints.items()])

## The scheduling problem

In [ ]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vQtBRpIr6Hx1_T0zJ3_DRqsE82YUjx7ZkeEKLdA64fbjtjkmc6Ibf6ebzp6CY69D482IGpG2h9GcsC5/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>